In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, IBMQ
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi
from utils import *


import grover, qaoa, ghz, qft, qft_entangled, shor, w_state, qpe_inexact, qpe_exact, graph_state

In [2]:
provider = IBMQ.load_account()
backend = provider.backend.ibmq_santiago
c_map = backend.configuration().coupling_map

In [3]:
num_qubits_test = 4

Grover Test

In [4]:
a = grover.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

QAOA Test

In [5]:
a = qaoa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

GHZ Test

In [6]:
a = ghz.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Test

In [7]:
a = qft.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Entangled Test

In [8]:
a = qft_entangled.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

W-State Test

In [9]:
a = w_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

QPE Exact and QPE Inexact Test

In [10]:
a = qpe_exact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

a = qpe_inexact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

Graph State Test

In [11]:
a = graph_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)


Shor Test

In [12]:
a = shor.create_circuit(15)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, None)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, None, True)

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Nils\anaconda3\envs\qsharp-env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized 


and got


but the original was native (for ['u1', 'cx', 'rx']) an

TranspilerError: 'Number of qubits (18) in shor_15_2 is greater than maximum (5) in the coupling_map'